In [1]:
import pandas as pd
import numpy as np
from os.path import join

# depending on your IDE, you might need to add datathon_eth. in front of data
from data import DataLoader, SimpleEncoding

# depending on your IDE, you might need to add datathon_eth. in front of forecast_models
from forecast_models import SimpleModel

country = "IT" 
"""

Train and evaluate the models for IT and ES

"""

# Inputs
input_path = r"datasets2025"
output_path = r"outputs"

# Load Datasets
loader = DataLoader(input_path)
# features are holidays and temperature
training_set, features, example_results = loader.load_data(country)

In [5]:
training_set.shape

(22629, 1696)

In [3]:
training_set.head()

,VALUEMWHMETERINGDATA_customerIT_1,VALUEMWHMETERINGDATA_customerIT_2,VALUEMWHMETERINGDATA_customerIT_3,VALUEMWHMETERINGDATA_customerIT_4,VALUEMWHMETERINGDATA_customerIT_5,VALUEMWHMETERINGDATA_customerIT_6,VALUEMWHMETERINGDATA_customerIT_7,VALUEMWHMETERINGDATA_customerIT_8,VALUEMWHMETERINGDATA_customerIT_13,VALUEMWHMETERINGDATA_customerIT_14,...,VALUEMWHMETERINGDATA_customerIT_2832,VALUEMWHMETERINGDATA_customerIT_2836,VALUEMWHMETERINGDATA_customerIT_2844,VALUEMWHMETERINGDATA_customerIT_2845,VALUEMWHMETERINGDATA_customerIT_2847,VALUEMWHMETERINGDATA_customerIT_2848,VALUEMWHMETERINGDATA_customerIT_2849,VALUEMWHMETERINGDATA_customerIT_2850,VALUEMWHMETERINGDATA_customerIT_2851,VALUEMWHMETERINGDATA_customerIT_2852
DATETIME,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,0.061939,NaN,0.032999,0.00330,NaN,...,NaN,NaN,NaN,0.41088,0.23192,0.06751,0.01971,0.117,0.00549,NaN
2022-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,0.057222,NaN,0.076501,0.00315,NaN,...,NaN,NaN,NaN,0.39106,0.23214,0.03488,0.01943,0.105,0.00551,NaN
2022-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,0.071429,NaN,0.129450,0.00315,NaN,...,NaN,NaN,NaN,0.40334,0.23323,0.04726,0.01971,0.108,0.00552,NaN
2022-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,0.064760,NaN,0.074399,0.00330,NaN,...,NaN,NaN,NaN,0.38904,0.23188,0.05513,0.01947,0.117,0.00551,NaN
2022-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,0.071744,NaN,0.054900,0.00315,NaN,...,NaN,NaN,NaN,0.41345,0.23478,0.03601,0.01981,0.108,0.00563,NaN


In [6]:
features

,spv,temp
2022-01-01 00:00:00,0.00000,3.9
2022-01-01 01:00:00,0.00000,3.7
2022-01-01 02:00:00,0.00000,3.4
2022-01-01 03:00:00,0.00000,3.4
2022-01-01 04:00:00,0.00000,3.6
...,...,...
2025-03-31 19:00:00,161.99736,13.1
2025-03-31 20:00:00,0.00000,11.4
2025-03-31 21:00:00,0.00000,10.3
2025-03-31 22:00:00,0.00000,9.2


In [9]:
holidays_path = "/Users/shyngys/personal/data-workspace/datasets2025/holiday_ES.xlsx"
date_format = "%Y-%m-%d %H:%M:%S"
holidays = pd.read_excel(
    holidays_path,
    # sheet_name=country,
    index_col=0,
    parse_dates=True,
    date_format=date_format,
)
holidays.head()

""
holiday_ES
2022-01-01
2022-01-06
2022-04-15
2022-08-15
2022-10-12


In [ ]:


"""
EVERYTHING STARTING FROM HERE CAN BE MODIFIED.
"""
team_name = "HANGUK_ML"
# Data Manipulation and Training
start_training = training_set.index.min()
end_training = training_set.index.max()
start_forecast, end_forecast = example_results.index[0], example_results.index[-1]

range_forecast = pd.date_range(start=start_forecast, end=end_forecast, freq="1H")
forecast = pd.DataFrame(columns=training_set.columns, index=range_forecast)
for costumer in training_set.columns.values:
    print(costumer)
    consumption = training_set.loc[:, costumer]
    
    feature_dummy = features['temp'].loc[start_training:]

    encoding = SimpleEncoding(
        consumption, feature_dummy, end_training, start_forecast, end_forecast
    )

    feature_past, feature_future, consumption_clean = (
        encoding.meta_encoding()
    )

    # Train
    model = SimpleModel()
    model.train(feature_past, consumption_clean)

    # Predict
    output = model.predict(feature_future)
    forecast[costumer] = output

"""
END OF THE MODIFIABLE PART.
"""

# test to make sure that the output has the expected shape.
dummy_error = np.abs(forecast - example_results).sum().sum()
assert np.all(forecast.columns == example_results.columns), (
    "Wrong header or header order."
)
assert np.all(forecast.index == example_results.index), (
    "Wrong index or index order."
)
assert isinstance(dummy_error, np.float64), "Wrong dummy_error type."
assert forecast.isna().sum().sum() == 0, "NaN in forecast."
# Your solution will be evaluated using
# forecast_error = np.abs(forecast - testing_set).sum().sum(),
# and then doing a weighted sum the two portfolios:
# score = forecast_error_IT + 5 * forecast_error_ES

forecast.to_csv(
    join(output_path, "students_results_" + team_name + "_" + country + ".csv")
)

